In [ ]:
import os, re, time, io, tarfile, logging, random, requests, xml.etree.ElementTree as ET
from typing import List, Dict, Optional, Callable
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import csv

EMAIL = "priyavejendla25@gmail.com"

OUT_DIR = "/content/drive/MyDrive/Glaucoma2"


TARGET_PMC  = 200   # non-TVST PMC glaucoma papers
TARGET_TVST = 150   # TVST glaucoma papers (from PMC)
TARGET_PDFS = TARGET_PMC + TARGET_TVST

PMC_QUERY  = "glaucoma"
TVST_JOURNAL_NAME = "Translational Vision Science & Technology"

# NCBI
PMC_RETMAX_PER_PAGE = 500
PMC_TOTAL_MAX = 40000
BATCH_SIZE_EFETCH = 50
MAX_RETRIES = 6
PDF_MIN_BYTES = 50_000
SLEEP_BASE = 0.8

EUTILS = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils"
PMC_ART = "https://www.ncbi.nlm.nih.gov/pmc/articles"

# ============== LOGGING & SESSION ==============
os.makedirs(OUT_DIR, exist_ok=True)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

SESSION = requests.Session()
SESSION.headers.update({"User-Agent": f"PMC-Glaucoma-TVST-Harvester/3.0 (mailto:{EMAIL})"})

retry_cfg = Retry(
    total=6,
    connect=6,
    read=6,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=frozenset(["GET"]),
    raise_on_status=False,
)
adapter = HTTPAdapter(max_retries=retry_cfg, pool_connections=50, pool_maxsize=50)
SESSION.mount("https://", adapter)
SESSION.mount("http://", adapter)


rows_lock = threading.Lock()
counters_lock = threading.Lock()

counters = {
    "checked_total": 0,
    "checked_pmc_general": 0,
    "checked_pmc_tvst": 0,

    "saved_total": 0,
    "saved_pmc_general": 0,
    "saved_pmc_tvst": 0,

    "skipped_nonhuman": 0,
    "skipped_no_pdf": 0,
    "skipped_duplicate": 0,
}

saved_pmcids = set()
saved_dois = set()
ROWS: List[Dict[str, str]] = []


def to_https_if_ftp(url: str) -> str:
    return url.replace("ftp://", "https://", 1) if url.startswith("ftp://") else url

def is_valid_pdf(data: bytes, content_type: Optional[str]) -> bool:
    if not data or len(data) < PDF_MIN_BYTES:
        return False
    if data[:5] == b"%PDF-":
        return True
    return bool(content_type and "application/pdf" in content_type.lower() and len(data) >= PDF_MIN_BYTES)

def jitter_sleep(base: float, factor: float = 0.4):
    time.sleep(base + random.random() * factor)

def safe_get(url: str, params: Optional[dict] = None, timeout: int = 45) -> requests.Response:
    url = to_https_if_ftp(url)
    last_err = None
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            r = SESSION.get(url, params=params, timeout=timeout, allow_redirects=True)
            if r.status_code == 200:
                return r
            if r.status_code in (429, 500, 502, 503, 504):
                wait = 0.8 * attempt
                logger.warning(f"{r.status_code} on {r.url or url}; retry in {wait:.1f}s")
                time.sleep(wait)
                continue
            r.raise_for_status()
        except Exception as e:
            last_err = e
            wait = 0.8 * attempt
            logger.warning(f"GET failed ({e}) for {url}; retry in {wait:.1f}s")
            time.sleep(wait)
    raise RuntimeError(f"HTTP failed after retries: {url} | {last_err}")

def chunked(seq, n):
    for i in range(0, len(seq), n):
        yield seq[i:i+n]

def clean_filename(title: str, fallback: str) -> str:
    safe = re.sub(r"[^a-zA-Z0-9_-]+", "_", (title or "").strip())[:80]
    return safe or fallback


def pmc_search_paged(term: str, total_max: int, page_size: int) -> List[str]:
    """Return numeric PMC IDs for a query."""
    ids: List[str] = []
    retstart = 0
    while retstart < total_max:
        params = {
            "email": EMAIL, "retmode": "xml", "db": "pmc",
            "term": term, "retmax": str(page_size), "retstart": str(retstart),
            "sort": "relevance",
        }
        r = safe_get(f"{EUTILS}/esearch.fcgi", params=params, timeout=40)
        root = ET.fromstring(r.text)
        page_ids = [x.text for x in root.findall(".//Id")]
        if not page_ids:
            break
        ids.extend(page_ids)
        logger.info(f"[PMC search] term='{term}' @ retstart={retstart}: +{len(page_ids)} (total {len(ids)})")
        retstart += page_size
        jitter_sleep(SLEEP_BASE)
        if len(ids) >= total_max:
            break
    return ids

def pmc_fetch_xml_batch(pmc_numeric_ids: List[str]) -> ET.Element:
    params = {"email": EMAIL, "retmode": "xml", "db": "pmc", "id": ",".join(pmc_numeric_ids)}
    r = safe_get(f"{EUTILS}/efetch.fcgi", params=params, timeout=60)
    return ET.fromstring(r.text)

# ============== HUMAN FILTER (PubMed MeSH) ==============
def pubmed_has_humans_mesh_by_pmid(pmid: str) -> bool:
    """True iff PubMed MeSH contains 'Humans' for this PMID."""
    try:
        params = {"email": EMAIL, "retmode": "xml", "db": "pubmed", "id": pmid}
        r = safe_get(f"{EUTILS}/efetch.fcgi", params=params, timeout=40)
        root = ET.fromstring(r.text)
        for mh in root.findall(".//MeshHeadingList/MeshHeading/DescriptorName"):
            if (mh.text or "").strip().lower() == "humans":
                return True
    except Exception as e:
        logger.info(f"[PubMed] MeSH check error for PMID {pmid}: {e}")
    return False

def is_human_article_strict_pmc(article: ET.Element) -> bool:
    """Use PMID if available; fallback to keywords if no PMID."""
    pmid_el = article.find(".//article-id[@pub-id-type='pmid']")
    if pmid_el is not None and pmid_el.text:
        pmid = pmid_el.text.strip()
        return pubmed_has_humans_mesh_by_pmid(pmid)

    texts = []
    for kw in article.findall(".//kwd"):
        texts.append("".join(kw.itertext()).strip().lower())
    for subj in article.findall(".//subj-group/subject"):
        texts.append("".join(subj.itertext()).strip().lower())
    for cat in article.findall(".//article-categories//subject"):
        texts.append("".join(cat.itertext()).strip().lower())
    tit_el = article.find(".//article-title")
    if tit_el is not None:
        texts.append("".join(tit_el.itertext()).strip().lower())
    whole = " ".join(texts)

    if "human" in whole or "humans" in whole:
        return True

    animal_cues = ["mouse", "mice", "murine", "rat", "rabbit", "porcine",
                   "canine", "feline", "zebrafish", "primate", "nonhuman", "non-human"]
    if any(cue in whole for cue in animal_cues):
        return False
    return True


def parse_pmc_meta(article: ET.Element) -> Dict[str, str]:
    title = "Unknown Title"
    tit_el = article.find(".//article-title")
    if tit_el is not None:
        title = "".join(tit_el.itertext()).strip()

    journal = "Unknown Journal"
    jour_el = article.find(".//journal-title")
    if jour_el is not None:
        journal = "".join(jour_el.itertext()).strip()

    pub_date = "N/A"
    pub_el = article.find(".//pub-date")
    if pub_el is not None:
        year = pub_el.findtext("year")
        month = pub_el.findtext("month")
        day = pub_el.findtext("day")
        if year:
            if month and day:
                pub_date = f"{year}-{month.zfill(2)}-{day.zfill(2)}"
            elif month:
                pub_date = f"{year}-{month.zfill(2)}-01"
            else:
                pub_date = year

    authors = []
    for au in article.findall(".//contrib-group/contrib[@contrib-type='author']"):
        last = au.findtext(".//surname") or ""
        fore = au.findtext(".//given-names") or ""
        name = f"{fore} {last}".strip()
        if name:
            authors.append(name)

    pmcid = ""
    pmid = ""
    doi = ""
    for aid in article.findall(".//article-id"):
        pid = aid.attrib.get("pub-id-type")
        if pid == "pmcid" and aid.text:
            pmcid = aid.text.strip()
        elif pid == "pmid" and aid.text:
            pmid = aid.text.strip()
        elif pid == "doi" and aid.text:
            doi = aid.text.strip()

    abstract = ""
    abs_el = article.find(".//abstract")
    if abs_el is not None:
        parts = []
        for p in abs_el.findall(".//p"):
            txt = "".join(p.itertext()).strip()
            if txt:
                parts.append(txt)
        abstract = "\n".join(parts) if parts else "".join(abs_el.itertext()).strip()

    return {
        "title": title or "Unknown Title",
        "journal": journal or "Unknown Journal",
        "pub_date": pub_date,
        "authors": "; ".join(authors),
        "pmcid": pmcid,
        "pmid": pmid,
        "doi": doi,
        "abstract": abstract,
    }

# PMC PDF DOWNLOAD
def oa_links(pmcid: str) -> Dict[str, str]:
    r = safe_get("https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi",
                 params={"id": pmcid}, timeout=30)
    links = {}
    root = ET.fromstring(r.text)
    for rec in root.findall(".//record"):
        for link in rec.findall("link"):
            fmt = link.attrib.get("format")
            href = link.attrib.get("href")
            if fmt and href:
                links[fmt] = to_https_if_ftp(href)
    return links

def parse_article_html_for_pdfs(pmcid: str) -> List[str]:
    art = f"{PMC_ART}/{pmcid}/"
    r = safe_get(art, timeout=45)
    soup = BeautifulSoup(r.text, "lxml")
    urls = []
    for a in soup.find_all("a", href=True):
        h = a["href"]
        if h.lower().endswith(".pdf"):
            urls.append(urljoin(art, h))
    seen = set()
    uniq = []
    for u in urls:
        if u not in seen:
            uniq.append(u)
            seen.add(u)
    return uniq

def download_pdf_pmc(pmcid: str) -> Optional[bytes]:
    # 1) OA 'pdf'
    try:
        links = oa_links(pmcid)
    except Exception as e:
        logger.info(f"[PMC] OA links error for {pmcid}: {e}")
        links = {}

    if "pdf" in links:
        try:
            r = safe_get(links["pdf"], timeout=45)
            if is_valid_pdf(r.content, r.headers.get("content-type")):
                return r.content
        except Exception:
            pass

    # 2) direct /pdf/ variants
    for u in (
        f"{PMC_ART}/{pmcid}/pdf/",
        f"{PMC_ART}/{pmcid}/pdf/?download=1",
        f"{PMC_ART}/{pmcid}/pdf/{pmcid}.pdf",
    ):
        try:
            r = safe_get(u, timeout=45)
            if is_valid_pdf(r.content, r.headers.get("content-type")):
                return r.content
        except Exception:
            pass

    # 3) pick largest *.pdf from HTML
    best = None
    best_size = -1
    for u in parse_article_html_for_pdfs(pmcid):
        try:
            rr = safe_get(u, timeout=60)
            if is_valid_pdf(rr.content, rr.headers.get("content-type")) and len(rr.content) > best_size:
                best = rr.content
                best_size = len(rr.content)
        except Exception:
            continue
    if best:
        return best

    # 4) OA .tgz → largest .pdf
    if "tgz" in links:
        try:
            r = safe_get(links["tgz"], timeout=60)
            buf = io.BytesIO(r.content)
            with tarfile.open(fileobj=buf, mode="r:gz") as tf:
                pdfs = [m for m in tf.getmembers()
                        if m.isfile() and m.name.lower().endswith(".pdf")]
                if pdfs:
                    pdfs.sort(key=lambda m: m.size, reverse=True)
                    data = tf.extractfile(pdfs[0]).read()
                    if is_valid_pdf(data, "application/pdf"):
                        return data
        except Exception:
            pass
    return None

# ============== SAVE ROW / PDF ==============
def save_pdf_and_row(pdf_data: bytes, title: str, source: str, meta: Dict[str, str]) -> Optional[str]:
    # Allocate index + update counters atomically (fix race condition)
    with counters_lock:
        counters["saved_total"] += 1
        next_idx = counters["saved_total"]
        if source == "PMC":
            counters["saved_pmc_general"] += 1
        elif source == "TVST-PMC":
            counters["saved_pmc_tvst"] += 1

    fname = f"{next_idx:03d}_{clean_filename(title, str(next_idx))}.pdf"
    path = os.path.join(OUT_DIR, fname)
    with open(path, "wb") as f:
        f.write(pdf_data)


    row = {
        "Title": meta.get("title",""),
        "Abstract": meta.get("abstract",""),
        "Authors": meta.get("authors",""),
        "Journal": meta.get("journal",""),
        "Pub_Date": meta.get("pub_date",""),
        "PMCID": meta.get("pmcid",""),
        "PMID": meta.get("pmid",""),
        "DOI": meta.get("doi",""),
        "Source": source,
        "PDF_Filename": fname,
    }
    with rows_lock:
        ROWS.append(row)

    print(f"Saved PDF #{next_idx}: {fname}", flush=True)
    return fname


def have_pmc_quota() -> bool:
    with counters_lock:
        return counters["saved_pmc_general"] < TARGET_PMC

def have_tvst_quota() -> bool:
    with counters_lock:
        return counters["saved_pmc_tvst"] < TARGET_TVST

# HARVESTERS
def harvest_pmc_general(stop_when: Callable[[], bool]):
    """
    General glaucoma PMC (excluding TVST journal by name).
    """
    pmc_ids = pmc_search_paged(PMC_QUERY, PMC_TOTAL_MAX, PMC_RETMAX_PER_PAGE)
    logger.info(f"[PMC-general] total IDs fetched: {len(pmc_ids)}")

    for batch in chunked(pmc_ids, BATCH_SIZE_EFETCH):
        if stop_when() or not have_pmc_quota():
            break
        try:
            root = pmc_fetch_xml_batch(batch)
        except Exception as e:
            logger.warning(f"[PMC-general] Batch efetch failed: {e}")
            jitter_sleep(SLEEP_BASE * 1.5)
            continue

        for art in root.findall(".//article"):
            if stop_when() or not have_pmc_quota():
                break

            pmcid_el = art.find(".//article-id[@pub-id-type='pmcid']")
            if pmcid_el is None or not pmcid_el.text:
                continue
            pmcid = pmcid_el.text.strip()

            meta = parse_pmc_meta(art)
            title = meta["title"]
            journal = (meta["journal"] or "").lower()
            doi = (meta.get("doi") or "").lower().strip()


            if "translational vision science & technology" in journal:
                continue

            with counters_lock:
                counters["checked_total"] += 1
                counters["checked_pmc_general"] += 1


            try:
                human_ok = is_human_article_strict_pmc(art)
            except Exception as e:
                logger.info(f"[PMC-general] Human check err on {pmcid}: {e}")
                human_ok = False
            if not human_ok:
                with counters_lock:
                    counters["skipped_nonhuman"] += 1
                continue


            with rows_lock:
                is_dup = pmcid in saved_pmcids or (doi and doi in saved_dois)
            if is_dup:
                with counters_lock:
                    counters["skipped_duplicate"] += 1
                continue

            pdf = None
            try:
                pdf = download_pdf_pmc(pmcid)
            except Exception as e:
                logger.info(f"[PMC-general] PDF fetch err {pmcid}: {e}")

            if not pdf:
                with counters_lock:
                    counters["skipped_no_pdf"] += 1
                continue

            save_pdf_and_row(pdf, title, "PMC", meta)

            with rows_lock:
                saved_pmcids.add(pmcid)
                if doi:
                    saved_dois.add(doi)

        jitter_sleep(SLEEP_BASE)

def harvest_pmc_tvst(stop_when: Callable[[], bool]):
    """
    TVST glaucoma papers via PMC: we search glaucoma,
    then filter by journal name = TVST.
    """
    pmc_ids = pmc_search_paged(PMC_QUERY, PMC_TOTAL_MAX, PMC_RETMAX_PER_PAGE)
    logger.info(f"[PMC-TVST] total IDs fetched: {len(pmc_ids)}")

    for batch in chunked(pmc_ids, BATCH_SIZE_EFETCH):
        if stop_when() or not have_tvst_quota():
            break
        try:
            root = pmc_fetch_xml_batch(batch)
        except Exception as e:
            logger.warning(f"[PMC-TVST] Batch efetch failed: {e}")
            jitter_sleep(SLEEP_BASE * 1.5)
            continue

        for art in root.findall(".//article"):
            if stop_when() or not have_tvst_quota():
                break

            pmcid_el = art.find(".//article-id[@pub-id-type='pmcid']")
            if pmcid_el is None or not pmcid_el.text:
                continue
            pmcid = pmcid_el.text.strip()

            meta = parse_pmc_meta(art)
            title = meta["title"]
            journal = (meta["journal"] or "").lower()
            doi = (meta.get("doi") or "").lower().strip()

            # Only keep TVST here
            if "translational vision science & technology" not in journal:
                continue

            with counters_lock:
                counters["checked_total"] += 1
                counters["checked_pmc_tvst"] += 1

            # strict human filter
            try:
                human_ok = is_human_article_strict_pmc(art)
            except Exception as e:
                logger.info(f"[PMC-TVST] Human check err on {pmcid}: {e}")
                human_ok = False
            if not human_ok:
                with counters_lock:
                    counters["skipped_nonhuman"] += 1
                continue

            # Check duplicates (pmcid or DOI)
            with rows_lock:
                is_dup = pmcid in saved_pmcids or (doi and doi in saved_dois)
            if is_dup:
                with counters_lock:
                    counters["skipped_duplicate"] += 1
                continue

            pdf = None
            try:
                pdf = download_pdf_pmc(pmcid)
            except Exception as e:
                logger.info(f"[PMC-TVST] PDF fetch err {pmcid}: {e}")

            if not pdf:
                with counters_lock:
                    counters["skipped_no_pdf"] += 1
                continue

            save_pdf_and_row(pdf, title, "TVST-PMC", meta)

            with rows_lock:
                saved_pmcids.add(pmcid)
                if doi:
                    saved_dois.add(doi)

        jitter_sleep(SLEEP_BASE)


def main():
    print(f"Targets -> PMC(non-TVST): {TARGET_PMC}, TVST: {TARGET_TVST}, TOTAL: {TARGET_PDFS}")

    def stop_when() -> bool:
        with counters_lock:
            return (counters["saved_pmc_general"] >= TARGET_PMC) and (counters["saved_pmc_tvst"] >= TARGET_TVST)

    with ThreadPoolExecutor(max_workers=2) as ex:
        fut_general = ex.submit(harvest_pmc_general, stop_when)
        fut_tvst    = ex.submit(harvest_pmc_tvst, stop_when)
        for fut in as_completed([fut_general, fut_tvst]):
            try:
                fut.result()
            except Exception as e:
                logger.error(f"Worker failed: {e}")


    csv_path = os.path.join(OUT_DIR, "glaucoma_pmc_tvst_only.csv")
    fieldnames = ["Title","Abstract","Authors","Journal","Pub_Date",
                  "PMCID","PMID","DOI","Source","PDF_Filename"]
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(ROWS)


    print("\n========== SUMMARY ==========")
    print(f"Checked total: {counters['checked_total']}")
    print(f"  - PMC general: {counters['checked_pmc_general']}")
    print(f"  - PMC TVST:    {counters['checked_pmc_tvst']}")
    print(f"Saved total:   {counters['saved_total']} / {TARGET_PDFS}")
    print(f"  - PMC general saved: {counters['saved_pmc_general']} / {TARGET_PMC}")
    print(f"  - TVST-PMC saved:    {counters['saved_pmc_tvst']} / {TARGET_TVST}")
    print(f"Skipped (non-human):   {counters['skipped_nonhuman']}")
    print(f"Skipped (no PDF):      {counters['skipped_no_pdf']}")
    print(f"Skipped (duplicate):   {counters['skipped_duplicate']}")
    print(f"CSV saved -> {csv_path}")
    print(f"PDFs saved in -> {OUT_DIR}")

if __name__ == "__main__":
    main()


Targets -> PMC(non-TVST): 200, TVST: 150, TOTAL: 350
Saved PDF #1: 001_Long-Term_Rate_of_Optic_Disc_Rim_Loss_in_Glaucoma_Patients_Measured_From_Optic_D.pdf
Saved PDF #2: 002_Neurodegeneration_Markers_Galectin-3_and_Apolipoprotein_E_Are_Elevated_in_the_Aq.pdf
Saved PDF #3: 003_Assessment_of_Causality_Between_Diet-Derived_Antioxidants_and_Primary_Open-Angle.pdf
Saved PDF #4: 004_Analysis_of_sociodemographic_profile_of_glaucoma_patients_with_risk_factors_subt.pdf
Saved PDF #5: 005_Molecular_Genetics_of_Glaucoma_Subtype_and_Ethnicity_Considerations.pdf
Saved PDF #6: 006_Artificial_Intelligence_Algorithms_to_Diagnose_Glaucoma_and_Detect_Glaucoma_Prog.pdf
Saved PDF #7: 007_Reduced_Aqueous_Humor_Outflow_Pathway_Arborization_in_Childhood_Glaucoma_Eyes.pdf


Saved PDF #8: 008_Artificial_Intelligence_for_Optical_Coherence_Tomography_in_Glaucoma.pdf


Saved PDF #9: 009_Contrast-to-Noise_Ratios_to_Evaluate_the_Detection_of_Glaucomatous_Progression_i.pdf
Saved PDF #10: 010_Central_Macular_Topographic_and_Volumetric_Measures_New_Biomarkers_for_Detection.pdf


Saved PDF #11: 011_The_OCT_RNFL_Probability_Map_and_Artifacts_Resembling_Glaucomatous_Damage.pdf


Saved PDF #12: 012_Individualized_Glaucoma_Change_Detection_Using_Deep_Learning_Auto_Encoder-Based_.pdf
Saved PDF #13: 013_Effects_of_Study_Population_Labeling_and_Training_on_Glaucoma_Detection_Using_De.pdf
Saved PDF #14: 014_Detecting_Established_Glaucoma_Using_OCT_Alone_Utilizing_an_OCT_Reading_Center_i.pdf


...

Saved PDF #331: 331_Widespread_retina_and_optic_nerve_neuroinflammation_in_enucleated_eyes_from_glau.pdf
Saved PDF #332: 332_Neuroprotection_in_glaucoma.pdf
Saved PDF #333: 333_The_Role_of_v_3_Integrin_in_Lamina_Cribrosa_Cell_Mechanotransduction_in_Glaucoma.pdf
Saved PDF #334: 334_Wound_modulation_in_glaucoma_surgery_The_role_of_anti-scarring_agents.pdf


Saved PDF #335: 335_Efficacy_of_dorzolamide_in_improving_ocular_blood_flow_in_patients_with_open-ang.pdf
Saved PDF #336: 336_The_association_between_glaucoma_and_all-cause_mortality_in_middle-aged_and_elde.pdf
Saved PDF #337: 337_Matrix_Metalloproteinases_and_Glaucoma_Treatment.pdf
Saved PDF #338: 338_Local_neuroplasticity_in_adult_glaucomatous_visual_cortex.pdf
Saved PDF #339: 339_Safety_and_efficacy_of_micropulse_in_pediatric_eyes_with_refractory_glaucoma.pdf
Saved PDF #340: 340_Childhood_glaucoma_profile_in_a_Southwestern_Ethiopia_tertiary_care_center_a_ret.pdf
Saved PDF #341: 341_Consensual_eye_intra-ocular_pressure_rise_following_unilateral_glaucoma_surgery.pdf
Saved PDF #342: 342_Outcomes_of_lensectomy_with_glued_intraocular_lens_and_factors_associated_with_f.pdf


Saved PDF #343: 343_OCT-based_diagnosis_of_glaucoma_and_glaucoma_stages_using_explainable_machine_le.pdf
Saved PDF #344: 344_Natural_History_and_Risk_Factors_for_Glaucoma_Progression_in_Chinese_Patients_Wi.pdf
Saved PDF #345: 345_Exploring_association_between_ambient_air_pollution_and_glaucoma_in_China_a_nati.pdf


Saved PDF #346: 346_A_deep-learning_system_predicts_glaucoma_incidence_and_progression_using_retinal.pdf


Saved PDF #347: 347_Identification_of_blood-derived_DNA_methylation_biomarkers_of_glaucoma_and_intra.pdf
Saved PDF #348: 348_Association_between_helicobacter_pylori_infection_and_primary_open-angle_glaucom.pdf
Saved PDF #349: 349_Proteomics_Analyses_of_Small_Extracellular_Vesicles_of_Aqueous_Humor_Identificat.pdf
Saved PDF #350: 350_The_pivotal_role_of_inflammatory_factors_in_glaucoma_a_systematic_review.pdf

========== SUMMARY ==========
Checked total: 522
  - PMC general: 331
  - PMC TVST:    191
Saved total:   350 / 350
  - PMC general saved: 200 / 200
  - TVST-PMC saved:    150 / 150
Skipped (non-human):   83
Skipped (no PDF):      89
Skipped (duplicate):   0
CSV saved -> /content/drive/MyDrive/Glaucoma2/glaucoma_pmc_tvst_only.csv
PDFs saved in -> /content/drive/MyDrive/Glaucoma2


Agent 2: Reader Agent

In [ ]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 39.9 MB/s eta 0:00:00


In [ ]:
import os
import re
import json
import fitz  # PyMuPDF
import csv
from typing import List, Dict


OUT_DIR = "/content/drive/MyDrive/Glaucoma2"
META_CSV = os.path.join(OUT_DIR, "glaucoma_pmc_tvst_only.csv")
SEGMENTS_DIR = os.path.join(OUT_DIR, "segments_agent2")
os.makedirs(SEGMENTS_DIR, exist_ok=True)


def load_metadata(csv_path: str) -> List[Dict[str, str]]:
    rows = []
    with open(csv_path, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        print("[INFO] Loaded", sum(1 for _ in open(csv_path, "r", encoding="utf-8")) - 1,
              "rows from", csv_path)
    with open(csv_path, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        print("[INFO] CSV columns:", reader.fieldnames)
        for r in reader:
            rows.append(r)
    return rows


def extract_pdf_text(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    texts = []
    for page in doc:
        texts.append(page.get_text("text"))
    doc.close()
    return "\n".join(texts)


SECTION_PATTERNS = {
    "abstract": re.compile(r"^\s*abstract\b", re.IGNORECASE),
    "introduction": re.compile(r"^\s*introduction\b", re.IGNORECASE),
    "methods": re.compile(r"^\s*(materials and methods|methods|patients and methods)\b",
                          re.IGNORECASE),
    "results": re.compile(r"^\s*results\b", re.IGNORECASE),
    "discussion": re.compile(r"^\s*discussion\b", re.IGNORECASE),
    "conclusion": re.compile(r"^\s*conclusions?\b", re.IGNORECASE),
}

def detect_section_heading(paragraph: str) -> str:
    p = paragraph.strip()
    for name, pat in SECTION_PATTERNS.items():
        if pat.match(p):
            return name.capitalize()
    return ""

def segment_text_into_sections(text: str) -> List[Dict[str, str]]:
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    raw_paragraphs = [p.strip() for p in re.split(r"\n\s*\n", text) if p.strip()]

    segments = []
    current_section = "Unknown"

    for para in raw_paragraphs:
        heading = detect_section_heading(para)
        if heading:
            current_section = heading
            continue

        segments.append({
            "section": current_section,
            "text": para
        })

    return segments


GLAUCOMA_KEYWORDS = [
    "glaucoma", "primary open-angle", "open-angle", "angle-closure",
    "normal-tension glaucoma", "ntg", "poag", "iog", "intraocular pressure",
    "iop", "optic nerve", "optic disc", "visual field", "rnfl",
    "retinal nerve fiber layer", "trabeculectomy", "latanoprost", "bimatoprost",
    "cup-to-disc", "cup disc", "perimetric", "humphrey", "oct", "ganglion cell"
]

def score_segment(text: str) -> float:
    t = text.lower()
    hits = 0
    for kw in GLAUCOMA_KEYWORDS:
        if kw in t:
            hits += 1
    if hits == 0:
        return 0.0
    return min(1.0, hits / 5.0)


def get_val(row: Dict[str, str], *names: str) -> str:
    """
    Try multiple possible column names (case insensitive) and return the first non-empty.
    Example: get_val(row, "pmcid", "PMCID")
    """
    lower_map = {k.lower(): v for k, v in row.items()}
    for name in names:
        v = lower_map.get(name.lower())
        if v is not None and str(v).strip() != "":
            return str(v).strip()
    return ""

#  MAIN: Agent 2 (Reader)
def run_agent2_reader():
    meta_rows = load_metadata(META_CSV)

    processed = 0
    missing_pmcid = 0
    missing_pdf = 0

    for row in meta_rows:
        pmcid = get_val(row, "pmcid", "PMCID")
        pdf_fname = get_val(row, "pdf_filename", "PDF_Filename")

        if not pmcid:
            missing_pmcid += 1
            continue
        if not pdf_fname:
            missing_pdf += 1
            continue

        pdf_path = os.path.join(OUT_DIR, pdf_fname)
        if not os.path.exists(pdf_path):
            print(f"[WARN] PDF not found for {pmcid}: {pdf_path}")
            missing_pdf += 1
            continue

        out_json = os.path.join(SEGMENTS_DIR, f"{pmcid}_segments.json")
        if os.path.exists(out_json):

            continue

        try:
            full_text = extract_pdf_text(pdf_path)
        except Exception as e:
            print(f"[ERROR] Failed to extract text for {pmcid}: {e}")
            continue

        segments_raw = segment_text_into_sections(full_text)
        segments = []
        for seg in segments_raw:
            score = score_segment(seg["text"])
            segments.append({
                "section": seg["section"],
                "text": seg["text"],
                "relevance_score": score,
            })

        data = {
            "pmcid": pmcid,
            "pdf_filename": pdf_fname,
            "segments": segments,
        }

        with open(out_json, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        processed += 1
        print(f"[Agent2] Processed {pmcid}, segments: {len(segments)}")

    print("\nAgent 2 finished.")
    print("  Processed papers :", processed)
    print("  Missing PMCID    :", missing_pmcid)
    print("  Missing PDFs     :", missing_pdf)
    print("Segments saved in  :", SEGMENTS_DIR)


if __name__ == "__main__":
    run_agent2_reader()


[INFO] Loaded 1356 rows from /content/drive/MyDrive/Glaucoma2/glaucoma_pmc_tvst_only.csv
[INFO] CSV columns: ['Title', 'Abstract', 'Authors', 'Journal', 'Pub_Date', 'PMCID', 'PMID', 'DOI', 'Source', 'PDF_Filename']
[Agent2] Processed PMC11379101, segments: 13
[Agent2] Processed PMC9639679, segments: 13
[Agent2] Processed PMC10910435, segments: 12
[Agent2] Processed PMC12013312, segments: 22
[Agent2] Processed PMC7823611, segments: 34
[Agent2] Processed PMC7571273, segments: 21
[Agent2] Processed PMC10981159, segments: 9
[Agent2] Processed PMC11760759, segments: 12
[Agent2] Processed PMC9123513, segments: 13
[Agent2] Processed PMC9344219, segments: 13
[Agent2] Processed PMC8934545, segments: 14
[Agent2] Processed PMC8300051, segments: 16
[Agent2] Processed PMC7396194, segments: 14
[Agent2] Processed PMC10777874, segments: 8
[Agent2] Processed PMC8802015, segments: 18
[Agent2] Processed PMC7424906, segments: 19
[Agent2] Processed PMC7488619, segments: 12
[Agent2] Processed PMC11645750, s